<a href="https://colab.research.google.com/github/nunsongi/quito-risk-mapping-python/blob/main/Limpieza_Sismos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Limpieza y Preparación de Datos Sísmicos de Quito 🇪🇨**
Este notebook tiene como objetivo cargar los catálogos sísmicos completos, filtrarlos para incluir solo los eventos dentro de la ciudad de Quito y guardar los resultados como un archivo CSV limpio para su posterior análisis.

## **Contenido**

1. Importación de librerías

2. Carga Inicial del Archivo TXT

3. Filtrado Geográfico (Quito)

4. Extender el Filtro a Magnitudes y Picks

5. Preprocesamiento y Limpieza Profunda

    5.1. Manejo de Duplicados (Inconsistencias)

    5.2. Conversión a Tipos de Datos Correctos (Parsing)

    * Conversión de Tiempo y Fecha (`datetime`)
    * Conversión de Valores Numéricos    
    
    5.3. Manejo de Valores Faltantes (NaN)

6. Revisión Final y Exportación a CSV

    6.1. Revisión Final

    6.2. Exportación a CSV

## 1. **Importación de librerías**

In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

## 2. **Carga Inicial del Archivo TXT**
Los archivos que tienen datos sobre los sismos se encuentran en formato .txt

In [ ]:
# Magnitud de los Sismos
columns_mag = [
    'event', 'orig_id', 'magnitude', 'magnitude_uncertainty',
    'magnitude_type', 'methodID', 'stationCount', 'agencyID'
]

archive_txt_mag = pd.read_csv(r'/content/cat_magnitud_2012-jul2025.txt',
                              sep='\,',
                              skiprows=27,
                              names=columns_mag,
                              encoding='utf-8')

print("Magnitud de los Sismos")
display(archive_txt_mag.head())

Magnitud de los Sismos


,event,orig_id,magnitude,magnitude_uncertainty,magnitude_type,methodID,stationCount,agencyID
0,igepn2012bafq,2766,3.6,0.33,MLv,mean,28,IG-EPN
1,igepn2012bafq,2766,3.6,NaN,M,weighted average,28,IG-EPN
2,igepn2012bbjo,2772,4.8,0.23,MLv,mean,14,IG-EPN
3,igepn2012bbjo,2772,4.8,NaN,M,weighted average,14,IG-EPN
4,igepn2012bhrl,2832,5.5,0.18,MLv,trimmed mean,27,IG-EPN


In [ ]:
# Origen de los Sismos
columns_origen = ['event', 'orig_id', 'time_value', 'time_value_ms', 'time_uncertainty',
                  'latitude_value', 'latitude_uncertainty', 'longitude_value',
                  'longitude_uncertainty', 'depth_value', 'depth_uncertainty',
                  'magnitude_value_M', 'magnitude_value_P', 'magnitude_type_P',
                  'magnitudeP_uncertainty', 'magnitudeP_stationCount',
                  'quality_associatedPhaseCount', 'quality_usedPhaseCount',
                  'quality_associatedStationCount', 'quality_usedStationCount',
                  'quality_standardError', 'quality_azimuthalGap', 'quality_maximumDistance',
                  'quality_minimumDistance', 'quality_medianDistance', 'Fuente', 'methodID' ,
                  'earthModelID']
archive_txt_origen = pd.read_csv(r'/content/cat_origen_2012-jul2025.txt',
                                 sep='\,',
                                 skiprows=54,
                                 names=columns_origen,
                                 encoding='utf-8')

print("Origen de los Sismos")
display(archive_txt_origen.head())

Origen de los Sismos


,event,orig_id,time_value,time_value_ms,time_uncertainty,latitude_value,latitude_uncertainty,longitude_value,longitude_uncertainty,depth_value,...,quality_associatedStationCount,quality_usedStationCount,quality_standardError,quality_azimuthalGap,quality_maximumDistance,quality_minimumDistance,quality_medianDistance,Fuente,methodID,earthModelID
0,igepn2012acxo,2611,2012-01-02 14:10:00.000,507680,0.740,-1.291955,6.01,-80.529152,4.74,12.00,...,31,23,0.85,233.55,319.54,44.93,237.95,IGEPN,LOCSAT,iasp91
1,igepn2012ahkn,2643,2012-01-05 01:18:17.000,293170,1.268,-1.822281,11.44,-81.669304,9.91,5.00,...,14,14,1.35,322.97,458.75,285.82,376.04,IGEPN,LOCSAT,tab
2,igepn2012ahzg,2647,2012-01-05 08:43:48.000,885316,1.454,-1.651550,10.68,-81.512100,9.98,10.00,...,21,19,0.88,320.57,441.64,260.26,353.72,IGEPN,LOCSAT,tab
3,igepn2012aigm,2649,2012-01-05 12:22:12.000,310021,0.415,0.886300,3.04,-78.776039,1.81,1.17,...,28,24,0.78,175.91,265.47,47.15,153.12,IGEPN,LOCSAT,iasp91
4,igepn2012ajib,2651,2012-01-06 02:16:44.000,929684,1.041,-1.675611,13.42,-81.597900,7.58,10.00,...,14,12,0.45,334.43,392.87,328.99,351.19,IGEPN,LOCSAT,tab


In [ ]:
# Tiempos de Onda de los Sismos
columns_picks = ['event', 'orig_id', 'time_value','time_value_ms','network_code','station_code',
                 'channel_code', 'phase_code', 'arri_azimuth', 'arri_distance', 'arri_timeResidual',
                 'arri_weight', 'polarity', 'time_used', 'agencyID'
]

archive_txt_picks = pd.read_csv(r'/content/cat_picks_2012-jul2025.txt',
                                sep='\,',
                                skiprows=33,
                                names=columns_picks,
                                encoding='utf-8')

print("Tiempos de Onda de los Sismos")
display(archive_txt_picks.head())

Tiempos de Onda de los Sismos


,event,orig_id,time_value,time_value_ms,network_code,station_code,channel_code,phase_code,arri_azimuth,arri_distance,arri_timeResidual,arri_weight,polarity,time_used,agencyID
0,igepn2012acvi,2610,2012-01-02 13:03:15.000,123425,EC,MAG1,SHZ,P,40.48,210.01,-1.30,1.0,,NaN,IGEPN
1,igepn2012acvi,2610,2012-01-02 13:03:21.000,921816,EC,IGUA,SHZ,P,89.33,261.97,-1.00,1.0,,NaN,IGEPN
2,igepn2012acvi,2610,2012-01-02 13:03:23.000,3382,EC,ILLI,HHZ,P,70.86,266.67,-0.50,1.0,,NaN,IGEPN
3,igepn2012acvi,2610,2012-01-02 13:03:24.000,393961,EC,POND,HHZ,P,87.85,282.83,-1.12,1.0,,NaN,IGEPN
4,igepn2012acvi,2610,2012-01-02 13:03:25.000,681536,EC,RETU,SHZ,P,88.52,283.43,0.09,1.0,,NaN,IGEPN


## 3. **Filtrado Geográfico (Quito)**

Nos enfocaremos solo en la ciudad capital (Quito)
Vamos a obtener los límites geográficos de la ciudad. Esto se hará en el DataFrame de Orígenes.


| Coordenada | Quito (Central) | Límite Mínimo | Límite Máximo |
| :--- | :--- | :--- | :--- |
| Latitud | $\approx -0.23$ | $\mathbf{-0.50}$ | $\mathbf{-0.05}$ |
|  | |  |  |
| Longitud | $\approx -78.52$ | $\mathbf{-78.80}$ | $\mathbf{-78.20}$ |

In [ ]:
# LÍMITES GEOGRÁFICOS PARA LA CIUDAD DE QUITO
LAT_MIN = -0.50
LAT_MAX = -0.05
LON_MIN = -78.80
LON_MAX = -78.20

# 1. Aplicar el filtro de ubicación al DataFrame de ORÍGENES
filtro_quito_origen = (
    (archive_txt_origen['latitude_value'] >= LAT_MIN) &
    (archive_txt_origen['latitude_value'] <= LAT_MAX) &
    (archive_txt_origen['longitude_value'] >= LON_MIN) &
    (archive_txt_origen['longitude_value'] <= LON_MAX)
)

df_origen_quito = archive_txt_origen[filtro_quito_origen].copy()

print(f"\nOrígenes de Sismos filtrados para la Ciudad de Quito: {len(df_origen_quito)}")
display(df_origen_quito.head())


Orígenes de Sismos filtrados para la Ciudad de Quito: 18


,event,orig_id,time_value,time_value_ms,time_uncertainty,latitude_value,latitude_uncertainty,longitude_value,longitude_uncertainty,depth_value,...,quality_associatedStationCount,quality_usedStationCount,quality_standardError,quality_azimuthalGap,quality_maximumDistance,quality_minimumDistance,quality_medianDistance,Fuente,methodID,earthModelID
537,igepn2014psys,27846,2014-08-12 19:57:58.000,932985,0.121,-0.050748,0.94,-78.418442,1.02,6.08,...,92,85,0.81,52.64,370.08,12.26,106.71,IGEPN-COTODP,LOCSAT,iasp91
538,igepn2014psyv,27847,2014-08-12 20:02:21.000,712717,0.159,-0.061485,0.99,-78.399765,1.26,3.46,...,59,56,0.51,40.53,159.92,13.99,69.86,IGEPN-COTODP,LOCSAT,iasp91
539,igepn2014ptck,27861,2014-08-12 21:49:55.000,874963,0.203,-0.064992,1.41,-78.425659,1.53,6.92,...,48,44,0.37,48.99,174.84,11.68,56.62,IGEPN-COTODP,LOCSAT,iasp91
542,igepn2014pund,27922,2014-08-13 16:23:20.000,713842,0.165,-0.058321,0.97,-78.419189,1.27,4.61,...,57,55,0.42,57.71,160.21,11.98,53.62,GFZ,LOCSAT,iasp91
543,igepn2014pvkq,27960,2014-08-14 04:16:11.000,77816,0.159,-0.057056,0.93,-78.432892,1.11,6.63,...,67,64,0.50,36.65,307.39,2.38,56.57,IGEPN-COTODP,LOCSAT,iasp91


## 4. **Extender el Filtro a Magnitudes y Picks**

Ahora que tenemos los eventos únicos de Quito, extraemos sus identificadores (`orig_id`) y los usamos para filtrar los otros dos DataFrames.

In [ ]:
# Obtener la lista de IDs de eventos que SÍ ocurrieron en Quito
ids_quito = df_origen_quito['orig_id'].unique()

# 2. Filtrar el DataFrame de MAGNITUDES
filtro_mag_quito = archive_txt_mag['orig_id'].isin(ids_quito)
df_mag_quito = archive_txt_mag[filtro_mag_quito].copy()

print(f"Registros de Magnitud filtrados para Quito: {len(df_mag_quito)}")
display(df_mag_quito.head())

Registros de Magnitud filtrados para Quito: 77


,event,orig_id,magnitude,magnitude_uncertainty,magnitude_type,methodID,stationCount,agencyID
1427,igepn2014pund,27922,3.9,0.48,MLv,mean,18,IG-EPN
1428,igepn2014pund,27922,3.7,NaN,M,weighted average,18,IG-EPN
4870,igepn2016pocw,103749,4.8,0.29,MLv,trimmed mean(25),37,IG-EPN
4871,igepn2016pocw,103749,4.5,0.17,Mjma,trimmed mean(25),33,IG-EPN
4872,igepn2016pocw,103749,4.7,NaN,M,weighted average,37,IG-EPN


In [ ]:
# Obtener la lista de IDs de eventos que SÍ ocurrieron en Quito
ids_quito = df_origen_quito['orig_id'].unique()

# 3. Filtrar el DataFrame de PICKS (Tiempos de Onda)
filtro_picks_quito = archive_txt_picks['orig_id'].isin(ids_quito)
df_picks_quito = archive_txt_picks[filtro_picks_quito].copy()

print(f"Registros de Tiempos de Onda (Picks) filtrados para Quito: {len(df_picks_quito)}")
display(df_picks_quito.head())

Registros de Tiempos de Onda (Picks) filtrados para Quito: 1514


,event,orig_id,time_value,time_value_ms,network_code,station_code,channel_code,phase_code,arri_azimuth,arri_distance,arri_timeResidual,arri_weight,polarity,time_used,agencyID
19436,igepn2014psys,27846,2014-08-12 19:58:01.000,963156,EC,PULU,HHZ,P,310.90,12.30,0.66,1.0,negative,NaN,IGEPN-COTODP
19437,igepn2014psys,27846,2014-08-12 19:58:02.000,189663,EC,PRAM,HNZ,P,219.01,13.45,0.71,1.0,,NaN,IGEPN-COTODP
19438,igepn2014psys,27846,2014-08-12 19:58:02.000,527927,EC,24MA,HNZ,P,203.70,15.29,0.76,1.0,,NaN,IGEPN-COTODP
19439,igepn2014psys,27846,2014-08-12 19:58:02.000,902377,EC,CIRC,HNZ,P,205.40,17.88,0.72,1.0,negative,NaN,IGEPN-COTODP
19440,igepn2014psys,27846,2014-08-12 19:58:02.000,927248,EC,CMEA,HNZ,P,206.04,17.97,0.73,1.0,negative,NaN,IGEPN-COTODP


## 5. **Preprocesamiento y Limpieza Profunda**

### 5.1. **Manejo de Duplicados (Inconsistencias)**
Es crucial asegurar que cada sismo sea único (`orig_id` o ` event`) aparezca una vez en las filas que deberían ser únicas (ej: orígenes).


In [ ]:
# Orígenes
# Un sismo debe tener solo 1 origen (lat/long/tiempo)

duplicados_origen = df_origen_quito.duplicated(keep=False)

# Contar cuántos eventos están duplicados
conteo_duplicados_origen = duplicados_origen.sum()

print(f"Número total de eventos sísmicos duplicados: {conteo_duplicados_origen}")

Número total de eventos sísmicos duplicados: 0


***Nota:*** No hay valores duplicados por lo tanto no hay nada que eliminar

En este caso, los archivos de Magnitudes y Tiempo de Ondas representan una medición de magnitud diferente y una estación sísmica diferente

Puede haber un error de la base de datos lo que genere una copia exacta de la fila, revisaremos eso ahora

In [ ]:
# Magnitudes

duplicados_mag = df_mag_quito.duplicated(keep=False)

# Contar cuántas filas están marcadas como duplicadas
conteo_duplicados_mag = duplicados_mag.sum()

print(f"Número total de filas duplicadas: {conteo_duplicados_mag}")

Número total de filas duplicadas: 0


In [ ]:
# Tiempo de Onda

# Crea una Serie Booleana
duplicados_picks = df_picks_quito.duplicated(keep=False)

# Contar cuántas filas están marcadas como duplicadas
conteo_duplicados_picks = duplicados_picks.sum()

print(f"\nNúmero total de filas duplicadas: {conteo_duplicados_picks}")


Número total de filas duplicadas: 0


***Nota:*** Tanto en Magnitudes como en Picks no hay filas duplicadas por lo tanto no hay nada que eliminar

### 5.2. **Conversión a Tipos de Datos Correctos (Parsing)**

Convertiremos las columnas de tiempo a `datetime` y las de valores numéricos (que a veces se importan como string u object) a `float`.

#### *  **Conversión de Tiempo y Fecha (`datetime`)**
Vamos a combinar `time_value` y `time_value_ms` para tener la fecha y hora completa con precisión de milisegundos.

In [ ]:
# Origenes

df_origen_quito['datetime'] = pd.to_datetime(
    df_origen_quito['time_value'],
    infer_datetime_format=True, # Pandas puede detectar el formato
    errors='coerce'             # Reemplaza los errores con NaT
)

print("Conversión de 'time_value' a 'datetime' completada.")
display(df_origen_quito[['time_value', 'datetime']].head())

Conversión de 'time_value' a 'datetime' completada.


,time_value,datetime
537,2014-08-12 19:57:58.000,2014-08-12 19:57:58
538,2014-08-12 20:02:21.000,2014-08-12 20:02:21
539,2014-08-12 21:49:55.000,2014-08-12 21:49:55
542,2014-08-13 16:23:20.000,2014-08-13 16:23:20
543,2014-08-14 04:16:11.000,2014-08-14 04:16:11


In [ ]:
# Tiempo de Ondas

df_picks_quito['datetime'] = pd.to_datetime(
    df_picks_quito['time_value'],
    infer_datetime_format=True,
    errors='coerce'
)

print("Conversión de 'time_value' a 'datetime' completada.")
display(df_picks_quito[['time_value', 'datetime']].head())

Conversión de 'time_value' a 'datetime' completada.


,time_value,datetime
19436,2014-08-12 19:58:01.000,2014-08-12 19:58:01
19437,2014-08-12 19:58:02.000,2014-08-12 19:58:02
19438,2014-08-12 19:58:02.000,2014-08-12 19:58:02
19439,2014-08-12 19:58:02.000,2014-08-12 19:58:02
19440,2014-08-12 19:58:02.000,2014-08-12 19:58:02


#### * **Conversión de Valores Numéricos**
Asegurar que todas las columnas de medición sean de tipo `float`.

In [ ]:
# Origen del Sismo
cols_to_numeric_origen = [
    'latitude_value', 'latitude_uncertainty', 'longitude_value', 'longitude_uncertainty',
    'depth_value', 'depth_uncertainty', 'magnitude_value_M', 'magnitude_value_P',
    'time_uncertainty', 'quality_standardError', 'quality_azimuthalGap',
    'quality_maximumDistance', 'quality_minimumDistance', 'quality_medianDistance'
]

for col in cols_to_numeric_origen:
    df_origen_quito[col] = pd.to_numeric(df_origen_quito[col], errors='coerce')

print("Conversión numérica en Origen de Sismo completada.")

Conversión numérica en Origen de Sismo completada.


In [ ]:
# Magnitud del Sismo
cols_to_numeric_mag = ['magnitude', 'magnitude_uncertainty', 'stationCount'
]

for col in cols_to_numeric_mag:
    df_mag_quito[col] = pd.to_numeric(df_mag_quito[col], errors='coerce')

print("Conversión numérica en Magnitud de Sismo completada.")

Conversión numérica en Magnitud de Sismo completada.


In [ ]:
# Tiempo de Onda
cols_to_numeric_picks = [
    'arri_distance', 'arri_azimuth', 'arri_timeResidual'
]

for col in cols_to_numeric_picks:
    df_picks_quito[col] = pd.to_numeric(df_picks_quito[col], errors='coerce')

print("Conversión numérica en Tiempo de Onda completada.")

Conversión numérica en Tiempo de Onda completada.


### 5.3. **Manejo de Valores Faltantes (NaN)**
Usaremos la combinación de los métodos `.isnull()`, `.sum()`, y un filtro simple de Pandas para mostrar solo las columnas que realmente tienen valores faltantes.

In [ ]:
# Origen del Sismo

print("Valores NaN en df_origen_quito")
nan_origen = df_origen_quito.isnull().sum()

# Filtra para mostrar solo las columnas con NaN > 0
print(nan_origen[nan_origen > 0].sort_values(ascending=False))

Valores NaN en df_origen_quito
time_uncertainty    2
dtype: int64


***Nota***: Origen del Sismo solo hay 2 valores nulos NaN

In [ ]:
# Magnitud de Sismo
print("Valores NaN en df_mag_quito")
nan_mag = df_mag_quito.isnull().sum()

# Filtra para mostrar solo las columnas con NaN > 0
print(nan_mag[nan_mag > 0].sort_values(ascending=False))

Valores NaN en df_mag_quito
magnitude_uncertainty    18
dtype: int64


***Nota:*** Magnitud de Sismo tiene 18 valores nulos

In [ ]:
# Tiempo de Onda
print("Valores NaN en df_picks_quito")
nan_picks = df_picks_quito.isnull().sum()

# Filtra para mostrar solo las columnas con NaN > 0
print(nan_picks[nan_picks > 0].sort_values(ascending=False))

Valores NaN en df_picks_quito
arri_timeResidual    20
dtype: int64


***Nota:*** Tiempo de Onda tiene 20 valores nulos

Manejaremos los valores nulos NaN de Origen de Sismo, de Magnitud y de Tiempo de Onda

Aplicaremos la estrategia de imputación (rellenar) para estas columnas, ya que son métricas secundarias

In [ ]:
# Rellenar 'time_uncertainty' con la mediana (el valor más representativo)

median_uncertainty = df_origen_quito['time_uncertainty'].median()
df_origen_quito['time_uncertainty'].fillna(median_uncertainty, inplace=True)

print(f"df_origen_quito: 'time_uncertainty' (2 NaN) rellenado con la mediana ({median_uncertainty:.3f}).")

# Verificamos
nan_conteo_origen = df_origen_quito['time_uncertainty'].isnull().sum()
print(f"\nConteo de NaN en 'time_uncertainty' después de la limpieza: {nan_conteo_origen}")

df_origen_quito: 'time_uncertainty' (2 NaN) rellenado con la mediana (0.151).

Conteo de NaN en 'time_uncertainty' después de la limpieza: 0


Efectivamente ya no hay valores nulos

In [ ]:
# Magnitud de Sismo
# Rellenar 'magnitude_uncertainty' con 0.
# La ausencia de incertidumbre a menudo significa que no se calculó, por lo que 0 es seguro aquí.
df_mag_quito['magnitude_uncertainty'].fillna(0, inplace=True)

print("df_mag_quito: 'magnitude_uncertainty' (18 NaN) rellenado con 0.")

# Verficamos
nan_conteo_mag = df_mag_quito['magnitude_uncertainty'].isnull().sum()
print(f"\nConteo de NaN en 'magnitude_uncertainty' después de la limpieza: {nan_conteo_mag}")

df_mag_quito: 'magnitude_uncertainty' (18 NaN) rellenado con 0.

Conteo de NaN en 'magnitude_uncertainty' después de la limpieza: 0


Los valores nulos fueron rellenados

In [ ]:
# Tiempo de Onda de Sismo
# Rellenar (arri_timeResidual) con la mediana.
# es la diferencia entre el tiempo de llegada de la onda observado en la estación
# y el tiempo de llegada predicho por el modelo. Es una métrica de calidad de la localización.
median_residual = df_picks_quito['arri_timeResidual'].median()
df_picks_quito['arri_timeResidual'].fillna(median_residual, inplace=True)

print(f"df_picks_quito: 'arri_timeResidual' (20 NaN) rellenado con la mediana ({median_residual:.4f}).")

# Verificamos
nan_conteo_picks = df_picks_quito['arri_timeResidual'].isnull().sum()
print(f"\nConteo de NaN en 'arri_timeResidual' después de la limpieza: {nan_conteo_picks}")

df_picks_quito: 'arri_timeResidual' (20 NaN) rellenado con la mediana (0.0100).

Conteo de NaN en 'arri_timeResidual' después de la limpieza: 0


## 6. **Revisión Final y Exportación a CSV**

### 6.1. **Revisión Final**
Una vez que la columna `datetime` está correcta, podemos eliminar las columnas `time_value` y `time_value_ms` para limpiar el espacio de trabajo.

In [ ]:
# ELIMINACIÓN FINAL DE COLUMNAS DE TIEMPO ORIGINALES

# df_origen_quito y df_picks_quito ya tienen 'datetime'
# Origen de sismo
df_origen_quito.drop(columns=['time_value', 'time_value_ms'], inplace=True, errors='ignore')

print("Columnas de tiempo originales eliminadas de Orígenes.")
display(df_origen_quito.head())

Columnas de tiempo originales eliminadas de Orígenes.


,event,orig_id,time_uncertainty,latitude_value,latitude_uncertainty,longitude_value,longitude_uncertainty,depth_value,depth_uncertainty,magnitude_value_M,...,quality_usedStationCount,quality_standardError,quality_azimuthalGap,quality_maximumDistance,quality_minimumDistance,quality_medianDistance,Fuente,methodID,earthModelID,datetime
537,igepn2014psys,27846,0.121,-0.050748,0.94,-78.418442,1.02,6.08,0.98,5.2,...,85,0.81,52.64,370.08,12.26,106.71,IGEPN-COTODP,LOCSAT,iasp91,2014-08-12 19:57:58
538,igepn2014psyv,27847,0.159,-0.061485,0.99,-78.399765,1.26,3.46,1.30,4.1,...,56,0.51,40.53,159.92,13.99,69.86,IGEPN-COTODP,LOCSAT,iasp91,2014-08-12 20:02:21
539,igepn2014ptck,27861,0.203,-0.064992,1.41,-78.425659,1.53,6.92,2.40,3.8,...,44,0.37,48.99,174.84,11.68,56.62,IGEPN-COTODP,LOCSAT,iasp91,2014-08-12 21:49:55
542,igepn2014pund,27922,0.165,-0.058321,0.97,-78.419189,1.27,4.61,1.08,3.7,...,55,0.42,57.71,160.21,11.98,53.62,GFZ,LOCSAT,iasp91,2014-08-13 16:23:20
543,igepn2014pvkq,27960,0.159,-0.057056,0.93,-78.432892,1.11,6.63,1.48,3.7,...,64,0.50,36.65,307.39,2.38,56.57,IGEPN-COTODP,LOCSAT,iasp91,2014-08-14 04:16:11


In [ ]:
# Tiempo de Onda de sismo
df_picks_quito.drop(columns=['time_value', 'time_value_ms'], inplace=True, errors='ignore')

print("Columnas de tiempo originales eliminadas de Picks.")
display(df_picks_quito.head())

Columnas de tiempo originales eliminadas de Picks.


,event,orig_id,network_code,station_code,channel_code,phase_code,arri_azimuth,arri_distance,arri_timeResidual,arri_weight,polarity,time_used,agencyID,datetime
19436,igepn2014psys,27846,EC,PULU,HHZ,P,310.90,12.30,0.66,1.0,negative,NaN,IGEPN-COTODP,2014-08-12 19:58:01
19437,igepn2014psys,27846,EC,PRAM,HNZ,P,219.01,13.45,0.71,1.0,,NaN,IGEPN-COTODP,2014-08-12 19:58:02
19438,igepn2014psys,27846,EC,24MA,HNZ,P,203.70,15.29,0.76,1.0,,NaN,IGEPN-COTODP,2014-08-12 19:58:02
19439,igepn2014psys,27846,EC,CIRC,HNZ,P,205.40,17.88,0.72,1.0,negative,NaN,IGEPN-COTODP,2014-08-12 19:58:02
19440,igepn2014psys,27846,EC,CMEA,HNZ,P,206.04,17.97,0.73,1.0,negative,NaN,IGEPN-COTODP,2014-08-12 19:58:02


### 6.2. **Exportación a CSV**
La limpieza ha terminado. El último paso es guardar los DataFrames.

In [ ]:
# EXPORTAR LOS TRES DATASETS DE QUITO LIMPIOS
df_origen_quito.to_csv('quito_origins.csv', index=False, encoding='utf-8')
df_mag_quito.to_csv('quito_magnitudes.csv', index=False, encoding='utf-8')
df_picks_quito.to_csv('quito_picks.csv', index=False, encoding='utf-8')

print("Los datasets están listos para el Análisis Exploratorio.")

Los datasets están listos para el Análisis Exploratorio.
